In [29]:
import pandas as pd
import plotly.express as px
import nbformat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import accuracy_score

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor




In [30]:
Dinning = pd.read_excel("DinningData.xlsx", sheet_name='Data for Linear')


In [31]:
Dinning.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          240 non-null    int64  
 1   Semester     240 non-null    int64  
 2   Month        240 non-null    int64  
 3   Year         240 non-null    int64  
 4   Hours        240 non-null    float64
 5   Money Spent  240 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 11.4 KB


In [32]:
Dinning

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [33]:
missing_values = Dinning.isnull().sum()
missing_values


DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [72]:
print(Dinning.columns)

Index(['DEP', 'Semester', 'Month', 'Year', 'Hours', 'Money Spent'], dtype='object')


In [80]:


def impute_with_models(df, target_column, feature_columns, n_neighbors=6,n_estimators=130, random_state=42):
    # Separate the data into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df.dropna(subset=[target_column])

    # If there are no missing values, return the original dataframe
    if df_missing.empty:
        print(f"No missing values for {target_column}. Skipping imputation.")
        return df, None, None, None, None, None, None, None, None  # Return None for metrics

    # Split df_not_missing into training and testing sets
    X = df_not_missing[feature_columns]
    y = df_not_missing[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit a linear regression model
    lr_model = LinearRegression()
    lr_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the linear regression model
    lr_y_pred = lr_model.predict(X_test)

    # Calculate evaluation metrics for linear regression
    lr_mse = mean_squared_error(y_test, lr_y_pred)
    lr_mae = mean_absolute_error(y_test, lr_y_pred)
    lr_r2 = r2_score(y_test, lr_y_pred)

    print(f"Linear Regression Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {lr_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {lr_mae:.2f}")
    print(f"R-squared (R²): {lr_r2:.2f}")

    # Create and fit a KNN model
    knn_model = KNeighborsRegressor(n_neighbors=n_neighbors)
    knn_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the KNN model
    knn_y_pred = knn_model.predict(X_test)

    # Calculate evaluation metrics for KNN
    knn_mse = mean_squared_error(y_test, knn_y_pred)
    knn_mae = mean_absolute_error(y_test, knn_y_pred)
    knn_r2 = r2_score(y_test, knn_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {knn_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {knn_mae:.2f}")
    print(f"R-squared (R²): {knn_r2:.2f}")

    # Create and fit a Random Forest Regressor model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
    rf_model.fit(X_train, y_train)

    # Predict on the test set to evaluate the model
    rf_y_pred = rf_model.predict(X_test)

    # Calculate evaluation metrics
    rf_mse = mean_squared_error(y_test, rf_y_pred)
    rf_mae = mean_absolute_error(y_test, rf_y_pred)
    rf_r2 = r2_score(y_test, rf_y_pred)

    print(f"KNN Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {rf_mse:.2f}")
    print(f"Mean Absolute Error (MAE): {rf_mae:.2f}")
    print(f"R-squared (R²): {rf_r2:.2f}")


    # Predict missing values using the model (choose the better model based on R²)
    X_missing = df_missing[feature_columns]
    if lr_r2 > knn_r2 and lr_r2 > rf_r2 :
        print("Using Linear Regression for imputation.")
        predicted_values = lr_model.predict(X_missing)
    elif knn_r2 > lr_r2 and knn_r2 > rf_r2:
        print("Using KNN for imputation.")
        predicted_values = knn_model.predict(X_missing)
    else:
        print("Using RF for imputation.")
        predicted_values = rf_model.predict(X_missing)

    # Create a dataset with imputed missing values
    df_filled = df.copy()
    df_filled.loc[df_missing.index, target_column] = predicted_values

    return df_filled, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2


In [81]:
target_columns_to_impute = ['Hours']
feature_columns_for_imputation = ['Year', 'Month', 'DEP', 'Semester']
linear_data = Dinning.copy()


In [82]:
for target_column in target_columns_to_impute:
    linear_data, lr_mse, lr_mae, lr_r2, knn_mse, knn_mae, knn_r2, rf_mse, rf_mae, rf_r2 = impute_with_models(linear_data, target_column, feature_columns_for_imputation)
    print(f"Linear Regression - MSE: {lr_mse}, MAE: {lr_mae}, R²: {lr_r2}")
    print(f"KNN - MSE: {knn_mse}, MAE: {knn_mae}, R²: {knn_r2}")
    print(f"RF - MSE: {rf_mse}, MAE: {rf_mae}, R²: {rf_r2}")

Linear Regression Evaluation metrics for Hours:
Mean Squared Error (MSE): 104433.28
Mean Absolute Error (MAE): 260.67
R-squared (R²): 0.47
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 27774.53
Mean Absolute Error (MAE): 123.58
R-squared (R²): 0.86
KNN Evaluation metrics for Hours:
Mean Squared Error (MSE): 43358.21
Mean Absolute Error (MAE): 121.01
R-squared (R²): 0.78
Using KNN for imputation.
Linear Regression - MSE: 104433.28144510179, MAE: 260.66824778571277, R²: 0.4678215644707766
KNN - MSE: 27774.526244753084, MAE: 123.5837037037037, R²: 0.8584646223889069
RF - MSE: 43358.20923415266, MAE: 121.00560897435886, R²: 0.7790521983194627


In [83]:
linear_data

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [84]:
missing_values = linear_data.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [85]:
df_2022 = linear_data[linear_data['Year'] == 2022]
df_2022


,DEP,Semester,Month,Year,Hours,Money Spent
16,1,2,1,2022,1297.593333,0.0
17,1,3,2,2022,1289.590000,0.0
18,1,3,3,2022,1455.546667,0.0
19,1,3,4,2022,1178.366667,0.0
20,1,3,5,2022,715.466667,0.0
21,1,4,6,2022,298.923333,0.0
22,1,4,7,2022,119.911667,0.0
23,1,4,8,2022,163.125000,0.0
24,1,1,9,2022,1689.936667,0.0
25,1,1,10,2022,1719.996667,0.0


In [87]:
CleanedDinningData = linear_data.copy()


In [88]:
CleanedDinningData

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
235,5,3,4,2024,119.09,1101.5825
236,5,3,5,2024,58.38,540.0150
237,5,4,6,2024,0.00,0.0000
238,5,4,7,2024,0.00,0.0000


In [41]:
CleanedDinningData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          240 non-null    int64  
 1   Semester     240 non-null    int64  
 2   Month        240 non-null    int64  
 3   Year         240 non-null    int64  
 4   Hours        240 non-null    float64
 5   Money Spent  240 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 11.4 KB


In [42]:
missing_values = CleanedDinningData.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64